In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
import time
from ctapipe.image import number_of_island_sizes
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates



final_rings = [5120, 8448, 9217, 25345, 26113, 20229, 36231, 20105, 22415, 9360, 12432, 31123, 35219, 5397, 33302, 31386, 17309, 19358, 19103, 27167, 33057, 33315, 21288, 27176, 8234, 28328, 34219, 6317, 34347, 35375, 6448, 26365, 20402, 32298, 26164, 33071, 21433, 9404, 22205, 9151, 32064, 6466, 14402, 34115, 14406, 25287, 31176, 3403, 29390, 29266, 9044, 17494, 32342, 25432, 9433, 9435, 22364, 22365, 25051, 28000, 36452, 12262, 26347, 35052, 35435, 16110, 22126, 27374, 22123, 22390, 16379, 20349, 3198]


In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0

t1 = time.perf_counter(), time.process_time()

#pixel_values = [1, 10, 20, 30, 40, 50, 70, 100, 150, 175, 200]
pixel_values = [100]
survived = []
len_container = []
r1_cut = 2


for pixel in pixel_values:
    counter = 0
    event_container = {}

    for k in range(1,41):
        filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
        source = EventSource(filename, max_events=9000)
        event_iterator = iter(source)
        keks = k*1000
        for i,j in enumerate(event_iterator):
            checker += 1
            event = j
            if len(j.trigger.tels_with_trigger) > 1:
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray)
                calib = CameraCalibrator(
                    image_extractor_type="GlobalPeakWindowSum",
                    subarray = source.subarray)
                for tel_id in j.trigger.tels_with_trigger:
                    tmp_arr = np.array([])
                    for pix_id in event.r1.tel[tel_id].waveform:
                        tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
                    pixel_number = len(tmp_arr[tmp_arr>r1_cut])
                    #print("pixel number = ", pixel_number)
                    if pixel_number > pixel:
                        event_container[keks+i] = j
    #                     calib(j)
    #                     image_processor(j)
    #                     muon_processor(j)
    #             for m in j.muon.tel.keys():
    #                 if not np.isnan(j.muon.tel[m].efficiency.width):
    #                     event_container[keks+i] = j
        
    for event in event_container:
        print("thats event = ", event)
        if event in final_rings:
            print("we are really counting")
            counter+=1
            
    survived.append(counter)
    len_container.append(len(event_container))
    print("survived = ", survived)
    print("len_cont = ", len_container)
    t2 = time.perf_counter(), time.process_time()   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
counter = 0
for event in event_container:
    #print("thats event = ", event)
    if event in final_rings:
        print("we are really counting")
        counter+=1

print(counter)
print(len(event_container))

In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0

t1 = time.perf_counter(), time.process_time()

pixel_values = [1, 20, 50, 60, 70, 80, 90, 100, 110, 120, 150, 175, 200]
#pixel_values = [100]
survived = []
len_container = []
r1_cut = 2


for pixel in pixel_values:
    counter = 0
    event_container = {}

    for k in range(1,41):
        filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
        source = EventSource(filename, max_events=9000)
        event_iterator = iter(source)
        keks = k*1000
        for i,j in enumerate(event_iterator):
            checker += 1
            event = j
            if len(j.trigger.tels_with_trigger) > 1:
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray)
                calib = CameraCalibrator(
                    image_extractor_type="GlobalPeakWindowSum",
                    subarray = source.subarray)
                for tel_id in j.trigger.tels_with_trigger:
                    tmp_arr = np.array([])
                    for pix_id in event.r1.tel[tel_id].waveform:
                        tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
                    pixel_number = len(tmp_arr[tmp_arr>r1_cut])
                    #print("pixel number = ", pixel_number)
                    if pixel_number > pixel:
                        event_container[keks+i] = j
    #                     calib(j)
    #                     image_processor(j)
    #                     muon_processor(j)
    #             for m in j.muon.tel.keys():
    #                 if not np.isnan(j.muon.tel[m].efficiency.width):
    #                     event_container[keks+i] = j
        
    for event in event_container:
        print("thats event = ", event)
        if event in final_rings:
            print("we are really counting")
            counter+=1
            
    survived.append(counter)
    len_container.append(len(event_container))
    print("survived = ", survived)
    print("len_cont = ", len_container)
    t2 = time.perf_counter(), time.process_time()   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
counter = 0
for event in event_container:
    #print("thats event = ", event)
    if event in final_rings:
        #print("we are really counting")
        counter+=1

print(counter)
print(len(event_container))

In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0

t1 = time.perf_counter(), time.process_time()

pixel_values = [1, 20, 50, 60, 70, 80, 90, 100, 110, 120, 150, 175, 200]
#pixel_values = [100]
survived = []
len_container = []
r1_cut = 1.5


for pixel in pixel_values:
    counter = 0
    event_container = {}

    for k in range(1,41):
        filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
        source = EventSource(filename, max_events=9000)
        event_iterator = iter(source)
        keks = k*1000
        for i,j in enumerate(event_iterator):
            checker += 1
            event = j
            if len(j.trigger.tels_with_trigger) > 1:
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray)
                calib = CameraCalibrator(
                    image_extractor_type="GlobalPeakWindowSum",
                    subarray = source.subarray)
                for tel_id in j.trigger.tels_with_trigger:
                    tmp_arr = np.array([])
                    for pix_id in event.r1.tel[tel_id].waveform:
                        tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
                    pixel_number = len(tmp_arr[tmp_arr>r1_cut])
                    #print("pixel number = ", pixel_number)
                    if pixel_number > pixel:
                        event_container[keks+i] = j
    #                     calib(j)
    #                     image_processor(j)
    #                     muon_processor(j)
    #             for m in j.muon.tel.keys():
    #                 if not np.isnan(j.muon.tel[m].efficiency.width):
    #                     event_container[keks+i] = j
        
    for event in event_container:
        print("thats event = ", event)
        if event in final_rings:
            print("we are really counting")
            counter+=1
            
    survived.append(counter)
    len_container.append(len(event_container))
    print("survived = ", survived)
    print("len_cont = ", len_container)
    t2 = time.perf_counter(), time.process_time()   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
counter = 0
for event in event_container:
    #print("thats event = ", event)
    if event in final_rings:
        #print("we are really counting")
        counter+=1

print(counter)
print(len(event_container))

In [ ]:
###
# Algorithm to flag muon rings
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0

t1 = time.perf_counter(), time.process_time()

pixel_values = [1, 20, 50, 60, 70, 80, 90, 100, 110, 120, 150, 175, 200]
#pixel_values = [100]
survived = []
len_container = []
r1_cut = 2.5


for pixel in pixel_values:
    counter = 0
    event_container = {}

    for k in range(1,41):
        filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
        source = EventSource(filename, max_events=9000)
        event_iterator = iter(source)
        keks = k*1000
        for i,event in enumerate(event_iterator):
            checker += 1
            if len(event.trigger.tels_with_trigger) > 1:
                for tel_id in event.trigger.tels_with_trigger:
                    tmp_arr = np.array([])
                    for pix_id in event.r1.tel[tel_id].waveform:
                        tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
                    pixel_number = len(tmp_arr[tmp_arr>r1_cut])
                    if pixel_number > pixel:
                        event_container[keks+i] = event
        
    for event in event_container:
        print("thats event = ", event)
        if event in final_rings:
            print("we are really counting")
            counter+=1
            
    survived.append(counter)
    len_container.append(len(event_container))
    print("survived = ", survived)
    print("len_cont = ", len_container)
    t2 = time.perf_counter(), time.process_time()   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
event_num = 2210
event = event_container[event_num]
print("tels with trigger = ", event.trigger.tels_with_trigger)
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray)
calib(event)
image_processor(event)
muon_processor(event)
print(event.dl1.tel[1].parameters.morphology.n_pixels)
print(event.dl1.tel[2].parameters.morphology.n_pixels)
print(event.dl1.tel[3].parameters.morphology.n_pixels)

In [ ]:
event_num = 2210
event = event_container[event_num]
tel = 1
r1_cut = 2
plt.figure(figsize = (9,6))
tmp_arr = np.array([])
n_sim_pix = np.count_nonzero(event.simulation.tel[tel].true_image)

for pix_id in event.r1.tel[tel].waveform:
    tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
pixel_number = len(tmp_arr[tmp_arr>r1_cut])

camgeom = source.subarray.tel[tel].camera.geometry
title=f"Event{event_num} - number of simulated pixels = {n_sim_pix} and Pixels with r1 waveform > 2 = {pixel_number}"
disp = CameraDisplay(camgeom,title=title)
disp.image = event.simulation.tel[tel].true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
plt.show()

In [ ]:
event.simulation.tel[1].true_image_sum

In [ ]:
event.simulation.shower.energy

In [ ]:
tel = 3
r1_cut = 2
plt.figure(figsize = (9,6))
tmp_arr = np.array([])
n_sim_pix = np.count_nonzero(event_container[2435].simulation.tel[tel].true_image)

for pix_id in event.r1.tel[tel].waveform:
    tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
pixel_number = len(tmp_arr[tmp_arr>r1_cut])

camgeom = source.subarray.tel[tel].camera.geometry
title=f"Event{event_num} - number of simulated pixels = {n_sim_pix} and Pixels with r1 waveform > 2 = {pixel_number}"
disp = CameraDisplay(camgeom,title=title)
disp.image = event_container[2435].simulation.tel[tel].true_image
disp.cmap = plt.cm.RdBu_r
disp.add_colorbar()
disp.set_limits_percent(95)
plt.show()

In [ ]:
event_container[2435].simulation.shower.energy

In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0

t1 = time.perf_counter(), time.process_time()

pixel_values = [1, 20, 50, 60, 70, 80, 90, 100, 110, 120, 150, 175, 200]
#pixel_values = [100]
survived = []
len_container = []
r1_cut = 3


for pixel in pixel_values:
    counter = 0
    event_container = {}

    for k in range(1,41):
        filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
        source = EventSource(filename, max_events=9000)
        event_iterator = iter(source)
        keks = k*1000
        for i,j in enumerate(event_iterator):
            checker += 1
            event = j
            if len(j.trigger.tels_with_trigger) > 1:
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray)
                calib = CameraCalibrator(
                    image_extractor_type="GlobalPeakWindowSum",
                    subarray = source.subarray)
                for tel_id in j.trigger.tels_with_trigger:
                    tmp_arr = np.array([])
                    for pix_id in event.r1.tel[tel_id].waveform:
                        tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
                    pixel_number = len(tmp_arr[tmp_arr>r1_cut])
                    #print("pixel number = ", pixel_number)
                    if pixel_number > pixel:
                        event_container[keks+i] = j
    #                     calib(j)
    #                     image_processor(j)
    #                     muon_processor(j)
    #             for m in j.muon.tel.keys():
    #                 if not np.isnan(j.muon.tel[m].efficiency.width):
    #                     event_container[keks+i] = j
        
    for event in event_container:
        print("thats event = ", event)
        if event in final_rings:
            print("we are really counting")
            counter+=1
            
    survived.append(counter)
    len_container.append(len(event_container))
    print("survived = ", survived)
    print("len_cont = ", len_container)
    t2 = time.perf_counter(), time.process_time()   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
counter = 0
for event in event_container:
    #print("thats event = ", event)
    if event in final_rings:
        #print("we are really counting")
        counter+=1

print(counter)
print(len(event_container))

In [ ]:
###
# Algorithm to process events and try to calibrate them
###
int_cut = 0.3
min_pixel = 80

muon_processor_config = Config({"MuonProcessor": 
                                {"RingQuery": 
                                 {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
                                                        ["ring_containment", "parameters.containment > 0.5"],
                                                        ["ring_completeness", "parameters.completeness > 0.5"]]}}})

#muon_processor_config = Config({"MuonProcessor":
#                                {"ImageParameterQuery" :
#                                {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

muon_processor_config_2 = Config({"MuonProcessor":
                                {"ImageParameterQuery" :
                                 {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
                                                        ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

checker = 0

t1 = time.perf_counter(), time.process_time()

pixel_values = [1, 20, 50, 60, 70, 80, 90, 100, 110, 120, 150, 175, 200]
#pixel_values = [100]
survived = []
len_container = []
r1_cut = 4


for pixel in pixel_values:
    counter = 0
    event_container = {}

    for k in range(1,41):
        filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
        source = EventSource(filename, max_events=9000)
        event_iterator = iter(source)
        keks = k*1000
        for i,j in enumerate(event_iterator):
            checker += 1
            event = j
            if len(j.trigger.tels_with_trigger) > 1:
                image_processor = ImageProcessor(source.subarray)
                muon_processor = MuonProcessor(source.subarray)
                calib = CameraCalibrator(
                    image_extractor_type="GlobalPeakWindowSum",
                    subarray = source.subarray)
                for tel_id in j.trigger.tels_with_trigger:
                    tmp_arr = np.array([])
                    for pix_id in event.r1.tel[tel_id].waveform:
                        tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
                    pixel_number = len(tmp_arr[tmp_arr>r1_cut])
                    #print("pixel number = ", pixel_number)
                    if pixel_number > pixel:
                        event_container[keks+i] = j
    #                     calib(j)
    #                     image_processor(j)
    #                     muon_processor(j)
    #             for m in j.muon.tel.keys():
    #                 if not np.isnan(j.muon.tel[m].efficiency.width):
    #                     event_container[keks+i] = j
        
    for event in event_container:
        print("thats event = ", event)
        if event in final_rings:
            print("we are really counting")
            counter+=1
            
    survived.append(counter)
    len_container.append(len(event_container))
    print("survived = ", survived)
    print("len_cont = ", len_container)
    t2 = time.perf_counter(), time.process_time()   

print(f" Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" CPU time: {t2[1] - t1[1]:.2f} seconds")
            
        

In [ ]:
counter = 0
for event in event_container:
    #print("thats event = ", event)
    if event in final_rings:
        #print("we are really counting")
        counter+=1

print(counter)
print(len(event_container))